# Bond Mechanics 202
<p style="color:darkblue; "><b>A TurningBull Notebook</b>

This notebook provides an advanced tutorial on bond yields and mechanics using Python examples. We'll explore U.S. Treasury bond pricing conventions, the relationship between bond prices and yields, and how to calculate total returns over various holding periods.

**Topics covered:**
1. U.S. Treasury Bond pricing conventions (1/32s pricing, premium vs discount bonds, clean vs dirty prices)
2. Bond price-yield relationship and different yield measures (nominal, current, yield to maturity)
3. Total return calculation for bonds held over various periods

Throughout this notebook, we'll use matplotlib to create visualizations that illustrate key concepts.

In [ ]:
## Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.optimize import newton
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. U.S. Treasury Bond Pricing Conventions

U.S. Treasury bonds are quoted in a unique format that differs from corporate bonds. Understanding these conventions is crucial for bond analysis.

### 1.1 Converting Quoted Prices from 1/32s to Dollar Amounts

Treasury bonds are quoted in points and 32nds of a point. For example, a quote of "101-16" means 101 and 16/32 points, which equals 101.50% of par value.

In [ ]:
def convert_treasury_quote(quote_str, par_value=1000):
    """
    Convert Treasury bond quote from 1/32s format to dollar price
    
    Parameters:
    quote_str: string in format "XXX-YY" where XXX is points and YY is 32nds
    par_value: face value of the bond (default $1000)
    
    Returns:
    Dollar price of the bond
    """
    if '-' in quote_str:
        points, thirty_seconds = quote_str.split('-')
        points = float(points)
        thirty_seconds = float(thirty_seconds)
    else:
        points = float(quote_str)
        thirty_seconds = 0
    
    # Convert to percentage
    percentage = points + (thirty_seconds / 32)
    
    # Convert to dollar amount
    dollar_price = (percentage / 100) * par_value
    
    return dollar_price, percentage

# Examples of Treasury quote conversions
quotes = ["100-00", "101-16", "99-24", "102-08", "98-12"]
print("Treasury Bond Quote Conversions:")
print("Quote\t\tDollar Price\tPercentage")
print("-" * 45)

for quote in quotes:
    dollar_price, percentage = convert_treasury_quote(quote)
    print(f"{quote}\t\t${dollar_price:.2f}\t\t{percentage:.3f}%")

### 1.2 Premium vs. Discount Bonds

- **Premium Bond**: Trades above par value (> 100% of face value)
- **Discount Bond**: Trades below par value (< 100% of face value)  
- **Par Bond**: Trades at face value (= 100% of face value)

The relationship between coupon rate and prevailing interest rates determines whether a bond trades at premium, discount, or par.

In [ ]:
# Create visualization showing premium vs discount bonds
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bond characteristics
coupon_rates = [2, 3, 4, 5, 6, 7, 8]
market_yield = 5.0  # Current market yield
par_value = 100
years_to_maturity = 10

def bond_price(coupon_rate, yield_rate, years, par=100):
    """Calculate bond price using present value formula"""
    coupon = coupon_rate * par / 100
    pv_coupons = sum([coupon / (1 + yield_rate/100)**t for t in range(1, years+1)])
    pv_principal = par / (1 + yield_rate/100)**years
    return pv_coupons + pv_principal

# Calculate bond prices
bond_prices = [bond_price(cr, market_yield, years_to_maturity) for cr in coupon_rates]

# Color code based on premium/discount
colors = ['red' if price < 100 else 'green' if price > 100 else 'blue' for price in bond_prices]
labels = ['Discount' if price < 100 else 'Premium' if price > 100 else 'Par' for price in bond_prices]

# First plot: Bond prices vs coupon rates
bars = ax1.bar(coupon_rates, bond_prices, color=colors, alpha=0.7)
ax1.axhline(y=100, color='black', linestyle='--', label='Par Value')
ax1.set_xlabel('Coupon Rate (%)')
ax1.set_ylabel('Bond Price ($)')
ax1.set_title(f'Bond Prices vs Coupon Rates\n(Market Yield: {market_yield}%)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add value labels on bars
for bar, price, label in zip(bars, bond_prices, labels):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'${price:.1f}\n{label}', ha='center', va='bottom', fontsize=9)

# Second plot: Price sensitivity to yield changes
yields = np.linspace(1, 10, 50)
prices_5pct = [bond_price(5, y, years_to_maturity) for y in yields]
prices_3pct = [bond_price(3, y, years_to_maturity) for y in yields]
prices_7pct = [bond_price(7, y, years_to_maturity) for y in yields]

ax2.plot(yields, prices_5pct, label='5% Coupon', linewidth=2)
ax2.plot(yields, prices_3pct, label='3% Coupon', linewidth=2)
ax2.plot(yields, prices_7pct, label='7% Coupon', linewidth=2)
ax2.axhline(y=100, color='black', linestyle='--', alpha=0.5, label='Par Value')
ax2.set_xlabel('Market Yield (%)')
ax2.set_ylabel('Bond Price ($)')
ax2.set_title('Bond Price Sensitivity to Yield Changes\n(10 Years to Maturity)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 1.3 Clean vs. Dirty Price and Accrued Interest

- **Clean Price**: The quoted price of a bond, excluding accrued interest
- **Dirty Price**: The actual price paid, including accrued interest
- **Accrued Interest**: Interest earned since the last coupon payment

**Formula**: Dirty Price = Clean Price + Accrued Interest

In [ ]:
def calculate_accrued_interest(coupon_rate, par_value, days_since_last_coupon, days_in_period, frequency=2):
    """
    Calculate accrued interest for a bond
    
    Parameters:
    coupon_rate: annual coupon rate (as percentage)
    par_value: face value of bond
    days_since_last_coupon: days since last coupon payment
    days_in_period: total days in coupon period
    frequency: coupon payments per year (default 2 for semi-annual)
    """
    annual_coupon = (coupon_rate / 100) * par_value
    period_coupon = annual_coupon / frequency
    accrued = period_coupon * (days_since_last_coupon / days_in_period)
    return accrued

# Example calculation
bond_params = {
    'coupon_rate': 4.5,
    'par_value': 1000,
    'clean_price': 102.25,  # as percentage of par
    'days_since_last_coupon': 45,
    'days_in_period': 182  # approximately 6 months
}

accrued_interest = calculate_accrued_interest(
    bond_params['coupon_rate'],
    bond_params['par_value'],
    bond_params['days_since_last_coupon'],
    bond_params['days_in_period']
)

clean_price_dollar = (bond_params['clean_price'] / 100) * bond_params['par_value']
dirty_price = clean_price_dollar + accrued_interest

print("Bond Pricing Example:")
print(f"Coupon Rate: {bond_params['coupon_rate']}%")
print(f"Par Value: ${bond_params['par_value']:,.2f}")
print(f"Clean Price: {bond_params['clean_price']}% = ${clean_price_dollar:,.2f}")
print(f"Days since last coupon: {bond_params['days_since_last_coupon']}")
print(f"Days in coupon period: {bond_params['days_in_period']}")
print(f"\nAccrued Interest: ${accrued_interest:.2f}")
print(f"Dirty Price: ${clean_price_dollar:,.2f} + ${accrued_interest:.2f} = ${dirty_price:,.2f}")

# Visualize accrued interest over a coupon period
days = np.arange(0, 183, 1)  # 6-month period
accrued_daily = [calculate_accrued_interest(4.5, 1000, day, 182) for day in days]

plt.figure(figsize=(12, 6))
plt.plot(days, accrued_daily, linewidth=2, color='blue')
plt.axvline(x=45, color='red', linestyle='--', label=f'Current day ({bond_params["days_since_last_coupon"]} days)')
plt.axhline(y=accrued_interest, color='red', linestyle='--', alpha=0.7)
plt.xlabel('Days Since Last Coupon Payment')
plt.ylabel('Accrued Interest ($)')
plt.title('Accrued Interest Over a 6-Month Coupon Period\n(4.5% Coupon, $1000 Par Value)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.text(45, accrued_interest + 1, f'${accrued_interest:.2f}', ha='center', va='bottom', 
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
plt.show()

## 2. Bond Price-Yield Relationship and Yield Measures

There are several ways to measure bond yields, each providing different insights into the investment's return potential.

### 2.1 Different Yield Measures

1. **Nominal Yield (Coupon Rate)**: The fixed interest rate stated on the bond
2. **Current Yield**: Annual coupon payment divided by current market price
3. **Yield to Maturity (YTM)**: The total return if held to maturity, accounting for all cash flows

In [ ]:
def calculate_current_yield(annual_coupon, current_price):
    """Calculate current yield"""
    return (annual_coupon / current_price) * 100

def bond_ytm_equation(ytm, coupon_rate, years_to_maturity, current_price, par_value=100, frequency=2):
    """
    Equation for YTM calculation - this should equal zero when YTM is correct
    """
    coupon_payment = (coupon_rate / 100) * par_value / frequency
    periods = years_to_maturity * frequency
    period_yield = ytm / frequency
    
    # Present value of coupon payments
    pv_coupons = sum([coupon_payment / (1 + period_yield)**t for t in range(1, periods + 1)])
    
    # Present value of principal
    pv_principal = par_value / (1 + period_yield)**periods
    
    return pv_coupons + pv_principal - current_price

def calculate_ytm_newton_raphson(coupon_rate, years_to_maturity, current_price, par_value=100, frequency=2, initial_guess=0.05):
    """
    Calculate YTM using Newton-Raphson method
    """
    try:
        ytm = newton(lambda y: bond_ytm_equation(y, coupon_rate, years_to_maturity, current_price, par_value, frequency), 
                    initial_guess, maxiter=100)
        return ytm * 100  # Convert to percentage
    except:
        return None

# Example bond for yield calculations
bond_example = {
    'coupon_rate': 5.0,
    'par_value': 100,
    'current_price': 98.50,
    'years_to_maturity': 8
}

# Calculate different yield measures
nominal_yield = bond_example['coupon_rate']
annual_coupon = (bond_example['coupon_rate'] / 100) * bond_example['par_value']
current_yield = calculate_current_yield(annual_coupon, bond_example['current_price'])
ytm = calculate_ytm_newton_raphson(
    bond_example['coupon_rate'],
    bond_example['years_to_maturity'],
    bond_example['current_price'],
    bond_example['par_value']
)

print("Yield Measure Comparison:")
print(f"Bond: {bond_example['coupon_rate']}% coupon, {bond_example['years_to_maturity']} years to maturity")
print(f"Current Price: ${bond_example['current_price']:.2f}")
print(f"Par Value: ${bond_example['par_value']:.2f}")
print("-" * 40)
print(f"Nominal Yield (Coupon Rate): {nominal_yield:.2f}%")
print(f"Current Yield: {current_yield:.2f}%")
print(f"Yield to Maturity: {ytm:.2f}%" if ytm else "YTM calculation failed")

# Create comparison chart
yield_types = ['Nominal Yield', 'Current Yield', 'YTM']
yield_values = [nominal_yield, current_yield, ytm if ytm else 0]

plt.figure(figsize=(10, 6))
bars = plt.bar(yield_types, yield_values, color=['lightblue', 'lightgreen', 'lightcoral'])
plt.ylabel('Yield (%)')
plt.title(f'Yield Comparison: {bond_example["coupon_rate"]}% Coupon Bond Trading at ${bond_example["current_price"]:.2f}')
plt.grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, yield_values):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             f'{value:.2f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

### 2.2 Newton-Raphson Method for YTM Calculation

The Newton-Raphson method is an iterative numerical technique used to find the roots of equations. For YTM calculation, we need to solve for the yield that makes the present value of all bond cash flows equal to the current market price.

In [ ]:
def ytm_newton_raphson_detailed(coupon_rate, years_to_maturity, current_price, par_value=100, frequency=2, 
                               initial_guess=0.05, tolerance=1e-6, max_iterations=100):
    """
    Detailed Newton-Raphson implementation for YTM with step-by-step tracking
    """
    def f(ytm):
        """Function to find root of (should equal zero)"""
        return bond_ytm_equation(ytm, coupon_rate, years_to_maturity, current_price, par_value, frequency)
    
    def f_prime(ytm, h=1e-8):
        """Numerical derivative"""
        return (f(ytm + h) - f(ytm)) / h
    
    ytm = initial_guess
    iterations = []
    
    for i in range(max_iterations):
        f_val = f(ytm)
        f_prime_val = f_prime(ytm)
        
        iterations.append({
            'iteration': i + 1,
            'ytm': ytm * 100,
            'f_value': f_val,
            'derivative': f_prime_val
        })
        
        if abs(f_val) < tolerance:
            break
            
        if abs(f_prime_val) < 1e-12:
            print("Warning: Derivative too small, may not converge")
            break
            
        ytm_new = ytm - f_val / f_prime_val
        ytm = ytm_new
    
    return ytm * 100, iterations

# Demonstrate Newton-Raphson for YTM calculation
ytm_result, iterations = ytm_newton_raphson_detailed(
    coupon_rate=4.75,
    years_to_maturity=10,
    current_price=96.50,
    initial_guess=0.06
)

print("Newton-Raphson Method for YTM Calculation")
print("Bond: 4.75% coupon, 10 years to maturity, trading at $96.50")
print("-" * 60)
print("Iter\tYTM (%)\t\tFunction Value\tDerivative")
print("-" * 60)

for iteration in iterations[:8]:  # Show first 8 iterations
    print(f"{iteration['iteration']}\t{iteration['ytm']:.4f}\t\t{iteration['f_value']:.6f}\t{iteration['derivative']:.4f}")

print(f"\nFinal YTM: {ytm_result:.4f}%")
print(f"Converged in {len(iterations)} iterations")

# Visualize convergence
ytm_values = [iter_data['ytm'] for iter_data in iterations]
f_values = [abs(iter_data['f_value']) for iter_data in iterations]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# YTM convergence
ax1.plot(range(1, len(ytm_values) + 1), ytm_values, 'bo-', linewidth=2, markersize=6)
ax1.axhline(y=ytm_result, color='red', linestyle='--', label=f'Final YTM: {ytm_result:.4f}%')
ax1.set_xlabel('Iteration')
ax1.set_ylabel('YTM (%)')
ax1.set_title('Newton-Raphson Convergence: YTM Estimates')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Function value convergence (log scale)
ax2.semilogy(range(1, len(f_values) + 1), f_values, 'ro-', linewidth=2, markersize=6)
ax2.set_xlabel('Iteration')
ax2.set_ylabel('|Function Value| (log scale)')
ax2.set_title('Newton-Raphson Convergence: Function Values')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2.3 Bond Yield Evolution: Premium, Discount, and Par Bonds

Let's examine how bond yields evolve as bonds approach maturity for different initial price scenarios.

In [ ]:
def simulate_bond_yield_evolution(coupon_rate, initial_price, years_to_maturity, par_value=100):
    """
    Simulate how bond yields change as the bond approaches maturity
    """
    time_points = np.linspace(years_to_maturity, 0, 50)
    yields = []
    prices = []
    
    for time_remaining in time_points:
        if time_remaining == 0:
            # At maturity, price equals par value
            price = par_value
            current_yield = coupon_rate  # At maturity, current yield equals coupon rate
        else:
            # Assume price converges to par as maturity approaches
            # Simple linear convergence model
            price = par_value + (initial_price - par_value) * (time_remaining / years_to_maturity)
            
            # Calculate current yield
            annual_coupon = (coupon_rate / 100) * par_value
            current_yield = (annual_coupon / price) * 100
        
        prices.append(price)
        yields.append(current_yield)
    
    return time_points, prices, yields

# Create three scenarios: Premium, Par, and Discount bonds
scenarios = {
    'Premium Bond': {'coupon_rate': 6.0, 'initial_price': 108.0, 'color': 'green'},
    'Par Bond': {'coupon_rate': 5.0, 'initial_price': 100.0, 'color': 'blue'},
    'Discount Bond': {'coupon_rate': 4.0, 'initial_price': 92.0, 'color': 'red'}
}

years = 10
par = 100

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot price evolution
for name, params in scenarios.items():
    time_points, prices, yields = simulate_bond_yield_evolution(
        params['coupon_rate'], params['initial_price'], years, par
    )
    
    ax1.plot(years - time_points, prices, label=f"{name} ({params['coupon_rate']}% coupon)", 
             color=params['color'], linewidth=2)

ax1.axhline(y=par, color='black', linestyle='--', alpha=0.5, label='Par Value')
ax1.set_xlabel('Years from Issue')
ax1.set_ylabel('Bond Price ($)')
ax1.set_title('Bond Price Evolution to Maturity')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot current yield evolution
for name, params in scenarios.items():
    time_points, prices, yields = simulate_bond_yield_evolution(
        params['coupon_rate'], params['initial_price'], years, par
    )
    
    ax2.plot(years - time_points, yields, label=f"{name} ({params['coupon_rate']}% coupon)", 
             color=params['color'], linewidth=2)

ax2.set_xlabel('Years from Issue')
ax2.set_ylabel('Current Yield (%)')
ax2.set_title('Current Yield Evolution to Maturity')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\nBond Scenarios Summary:")
print("Bond Type\t\tCoupon\tInitial Price\tAt Maturity")
print("-" * 55)
for name, params in scenarios.items():
    initial_yield = (params['coupon_rate'] * par / params['initial_price'])
    print(f"{name:<15}\t{params['coupon_rate']:.1f}%\t${params['initial_price']:.2f}\t\t${par:.2f}")

## 3. Total Return Calculation for Bonds

Total return measures the complete return from holding a bond over a specific period, including both income and capital appreciation/depreciation.

### 3.1 Components of Bond Return

Bond total return consists of three main components:

1. **Coupon Income**: Interest payments received during the holding period
2. **Capital Gain/Loss**: Change in bond price from purchase to sale
3. **Reinvestment Income**: Return from reinvesting coupon payments (if applicable)

**Formula**: Total Return = (Coupon Income + Capital Gain/Loss + Reinvestment Income) / Initial Investment

In [ ]:
def calculate_bond_total_return(purchase_price, sale_price, coupon_rate, par_value, 
                               holding_period_years, reinvestment_rate=None, frequency=2):
    """
    Calculate total return for a bond held over a specific period
    
    Parameters:
    purchase_price: price paid for bond
    sale_price: price received when sold
    coupon_rate: annual coupon rate (as percentage)
    par_value: face value of bond
    holding_period_years: years bond was held
    reinvestment_rate: rate at which coupons are reinvested (if None, assumes no reinvestment)
    frequency: coupon payments per year
    """
    
    # Calculate coupon income
    annual_coupon = (coupon_rate / 100) * par_value
    total_coupon_payments = annual_coupon * holding_period_years
    
    # Calculate capital gain/loss
    capital_gain_loss = sale_price - purchase_price
    
    # Calculate reinvestment income (if applicable)
    reinvestment_income = 0
    if reinvestment_rate is not None:
        coupon_per_period = annual_coupon / frequency
        periods = int(holding_period_years * frequency)
        period_reinvestment_rate = reinvestment_rate / frequency
        
        # Future value of reinvested coupons
        fv_coupons = 0
        for period in range(periods):
            periods_remaining = periods - period - 1
            fv_coupons += coupon_per_period * (1 + period_reinvestment_rate) ** periods_remaining
        
        reinvestment_income = fv_coupons - total_coupon_payments
    
    # Total return components
    total_dollar_return = total_coupon_payments + capital_gain_loss + reinvestment_income
    total_return_percentage = (total_dollar_return / purchase_price) * 100
    
    # Annualized return
    annualized_return = ((1 + total_return_percentage/100) ** (1/holding_period_years) - 1) * 100
    
    return {
        'coupon_income': total_coupon_payments,
        'capital_gain_loss': capital_gain_loss,
        'reinvestment_income': reinvestment_income,
        'total_dollar_return': total_dollar_return,
        'total_return_pct': total_return_percentage,
        'annualized_return': annualized_return
    }

# Example: Bond held for different periods
bond_data = {
    'coupon_rate': 5.5,
    'par_value': 1000,
    'purchase_price': 980,
    'reinvestment_rate': 0.04  # 4% reinvestment rate
}

# Different holding periods and sale prices
scenarios = [
    {'period': 1, 'sale_price': 985, 'description': '1 Year Hold'},
    {'period': 3, 'sale_price': 995, 'description': '3 Year Hold'},
    {'period': 5, 'sale_price': 1000, 'description': '5 Year Hold (to maturity)'},
    {'period': 2, 'sale_price': 975, 'description': '2 Year Hold (price decline)'}
]

print("Bond Total Return Analysis")
print(f"Bond: {bond_data['coupon_rate']}% coupon, purchased at ${bond_data['purchase_price']}")
print(f"Reinvestment rate: {bond_data['reinvestment_rate']*100}%")
print("=" * 80)

results = []
for scenario in scenarios:
    result = calculate_bond_total_return(
        bond_data['purchase_price'],
        scenario['sale_price'],
        bond_data['coupon_rate'],
        bond_data['par_value'],
        scenario['period'],
        bond_data['reinvestment_rate']
    )
    result['description'] = scenario['description']
    result['period'] = scenario['period']
    result['sale_price'] = scenario['sale_price']
    results.append(result)
    
    print(f"\n{scenario['description']} - Sale Price: ${scenario['sale_price']}")
    print(f"  Coupon Income: ${result['coupon_income']:.2f}")
    print(f"  Capital Gain/Loss: ${result['capital_gain_loss']:.2f}")
    print(f"  Reinvestment Income: ${result['reinvestment_income']:.2f}")
    print(f"  Total Dollar Return: ${result['total_dollar_return']:.2f}")
    print(f"  Total Return: {result['total_return_pct']:.2f}%")
    print(f"  Annualized Return: {result['annualized_return']:.2f}%")

# Create visualization of return components
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Stacked bar chart of return components
descriptions = [r['description'] for r in results]
coupon_income = [r['coupon_income'] for r in results]
capital_gains = [r['capital_gain_loss'] for r in results]
reinvestment = [r['reinvestment_income'] for r in results]

x = np.arange(len(descriptions))
width = 0.6

p1 = ax1.bar(x, coupon_income, width, label='Coupon Income', color='lightblue')
p2 = ax1.bar(x, capital_gains, width, bottom=coupon_income, label='Capital Gain/Loss', color='lightgreen')
p3 = ax1.bar(x, reinvestment, width, bottom=np.array(coupon_income) + np.array(capital_gains), 
            label='Reinvestment Income', color='lightyellow')

ax1.set_ylabel('Dollar Return ($)')
ax1.set_title('Bond Return Components by Holding Period')
ax1.set_xticks(x)
ax1.set_xticklabels([d.replace(' ', '\n') for d in descriptions])
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add value labels
for i, (c, g, r) in enumerate(zip(coupon_income, capital_gains, reinvestment)):
    total = c + g + r
    ax1.text(i, total + 5, f'${total:.0f}', ha='center', va='bottom', fontweight='bold')

# Annualized returns comparison
annualized_returns = [r['annualized_return'] for r in results]
colors = ['green' if ret > 0 else 'red' for ret in annualized_returns]

bars = ax2.bar(x, annualized_returns, width, color=colors, alpha=0.7)
ax2.set_ylabel('Annualized Return (%)')
ax2.set_title('Annualized Returns by Holding Period')
ax2.set_xticks(x)
ax2.set_xticklabels([d.replace(' ', '\n') for d in descriptions])
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)

# Add percentage labels
for bar, ret in zip(bars, annualized_returns):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + (0.1 if height > 0 else -0.3),
             f'{ret:.2f}%', ha='center', va='bottom' if height > 0 else 'top', fontweight='bold')

plt.tight_layout()
plt.show()

### 3.2 Impact of Holding Period on Total Return

Let's examine how holding period affects total return for bonds with different characteristics.

In [ ]:
def analyze_holding_period_impact(coupon_rate, purchase_price, par_value=1000, max_years=10):
    """
    Analyze how total return changes with different holding periods
    """
    holding_periods = np.arange(0.5, max_years + 0.5, 0.5)
    
    # Assume different sale price scenarios
    scenarios = {
        'Stable Yield Environment': lambda years: par_value + (purchase_price - par_value) * np.exp(-0.15 * years),
        'Rising Yield Environment': lambda years: purchase_price - 5 * years,  # Price declines over time
        'Falling Yield Environment': lambda years: purchase_price + 3 * years   # Price increases over time
    }
    
    results = {}
    
    for scenario_name, price_function in scenarios.items():
        annualized_returns = []
        total_returns = []
        
        for period in holding_periods:
            sale_price = max(price_function(period), par_value * 0.7)  # Floor at 70% of par
            sale_price = min(sale_price, par_value * 1.3)  # Ceiling at 130% of par
            
            result = calculate_bond_total_return(
                purchase_price, sale_price, coupon_rate, par_value, period, 0.03
            )
            
            annualized_returns.append(result['annualized_return'])
            total_returns.append(result['total_return_pct'])
        
        results[scenario_name] = {
            'annualized_returns': annualized_returns,
            'total_returns': total_returns
        }
    
    return holding_periods, results

# Analyze for a typical bond
periods, analysis_results = analyze_holding_period_impact(
    coupon_rate=4.5,
    purchase_price=975,
    par_value=1000
)

# Create comprehensive visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

colors = ['blue', 'red', 'green']
linestyles = ['-', '--', ':']

# Annualized returns by holding period
for i, (scenario_name, data) in enumerate(analysis_results.items()):
    ax1.plot(periods, data['annualized_returns'], 
             color=colors[i], linestyle=linestyles[i], linewidth=2, 
             label=scenario_name, marker='o', markersize=4)

ax1.set_xlabel('Holding Period (Years)')
ax1.set_ylabel('Annualized Return (%)')
ax1.set_title('Annualized Returns vs Holding Period\n(4.5% Coupon Bond, Purchased at $975)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.axhline(y=0, color='black', linestyle='-', alpha=0.3)

# Total returns by holding period
for i, (scenario_name, data) in enumerate(analysis_results.items()):
    ax2.plot(periods, data['total_returns'], 
             color=colors[i], linestyle=linestyles[i], linewidth=2, 
             label=scenario_name, marker='s', markersize=4)

ax2.set_xlabel('Holding Period (Years)')
ax2.set_ylabel('Total Return (%)')
ax2.set_title('Total Returns vs Holding Period')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)

# Return attribution analysis for stable environment
stable_periods = [1, 3, 5, 7, 10]
coupon_contributions = []
capital_contributions = []

for period in stable_periods:
    # Simulate stable environment
    sale_price = 1000 + (975 - 1000) * np.exp(-0.15 * period)
    result = calculate_bond_total_return(975, sale_price, 4.5, 1000, period, 0.03)
    
    total_return = result['total_dollar_return']
    coupon_pct = (result['coupon_income'] + result['reinvestment_income']) / 975 * 100
    capital_pct = result['capital_gain_loss'] / 975 * 100
    
    coupon_contributions.append(coupon_pct)
    capital_contributions.append(capital_pct)

x = np.arange(len(stable_periods))
width = 0.6

p1 = ax3.bar(x, coupon_contributions, width, label='Income Return', color='lightblue')
p2 = ax3.bar(x, capital_contributions, width, bottom=coupon_contributions, 
            label='Capital Return', color='lightcoral')

ax3.set_ylabel('Return Contribution (%)')
ax3.set_xlabel('Holding Period (Years)')
ax3.set_title('Return Attribution: Income vs Capital\n(Stable Yield Environment)')
ax3.set_xticks(x)
ax3.set_xticklabels(stable_periods)
ax3.legend()
ax3.grid(True, alpha=0.3)

# Risk-return profile
# Calculate volatility of returns across scenarios for each period
return_volatilities = []
average_returns = []

for i, period in enumerate(periods):
    period_returns = [data['annualized_returns'][i] for data in analysis_results.values()]
    return_volatilities.append(np.std(period_returns))
    average_returns.append(np.mean(period_returns))

ax4.scatter(return_volatilities, average_returns, c=periods, cmap='viridis', s=60, alpha=0.7)
cbar = plt.colorbar(ax4.scatter(return_volatilities, average_returns, c=periods, cmap='viridis', s=60, alpha=0.7), ax=ax4)
cbar.set_label('Holding Period (Years)')
ax4.set_xlabel('Return Volatility (Standard Deviation %)')
ax4.set_ylabel('Average Annualized Return (%)')
ax4.set_title('Risk-Return Profile by Holding Period')
ax4.grid(True, alpha=0.3)

# Add annotations for specific periods
for i, period in enumerate([1, 5, 10]):
    if period in periods:
        idx = list(periods).index(period)
        ax4.annotate(f'{period}Y', 
                    (return_volatilities[idx], average_returns[idx]),
                    xytext=(5, 5), textcoords='offset points', fontsize=9)

plt.tight_layout()
plt.show()

print("Key Observations:")
print("1. Longer holding periods generally reduce the impact of short-term price volatility")
print("2. In stable yield environments, returns converge toward the yield to maturity")
print("3. Income return (coupons) becomes more significant with longer holding periods")
print("4. The risk-return profile varies significantly with holding period and market conditions")

## Summary

This notebook has covered the essential aspects of bond mechanics and yield analysis:

### Key Takeaways:

1. **Pricing Conventions**: U.S. Treasury bonds use unique 1/32s pricing, and understanding clean vs. dirty prices is crucial for accurate analysis.

2. **Yield Measures**: Different yield measures serve different purposes:
   - Nominal yield provides the fixed rate
   - Current yield shows income relative to current price
   - YTM provides total return if held to maturity

3. **Newton-Raphson Method**: An efficient numerical technique for calculating YTM when analytical solutions are impractical.

4. **Total Return Analysis**: Understanding all components (coupon income, capital gains/losses, reinvestment income) is essential for comprehensive bond evaluation.

5. **Holding Period Impact**: The relationship between holding period, market conditions, and total return is complex and requires careful analysis.

### Practical Applications:

- Portfolio managers use these concepts for bond selection and risk management
- Individual investors can better understand bond behavior in different market environments
- Risk managers utilize these tools for duration and convexity analysis
- Traders apply these principles for identifying arbitrage opportunities

Understanding these bond mechanics provides the foundation for more advanced fixed income analysis, including duration, convexity, and complex derivative instruments.